In [23]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from matplotlib import pyplot as plt
import time
import pandas as pd

In [24]:
with open('../input.txt','r',encoding='utf-8') as f:
    text= f.read()

In [25]:
print("Length",len(text))

Length 1115394


In [26]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [27]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [56]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stoi[c] for c in s]
decode=lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [48]:
print(stoi)
print(itos)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [49]:
import torch
data= torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)


torch.Size([1115394]) torch.int64


In [50]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [51]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [52]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [53]:
torch.manual_seed(1377)
batch_size=4
block_size=8

def get_batch(split):
    data=train_data if split== 'train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context=xb[b, :t+1]
        target=yb[b,t]
        print(f"when input is {context.tolist()} the target:{target}")

inputs:
torch.Size([4, 8])
tensor([[58, 46, 43,  1, 58, 47, 51, 43],
        [50, 50,  1, 51, 43,  1, 52, 53],
        [57,  1, 49, 52, 53, 58,  1, 49],
        [59, 45, 46, 58, 43, 56,  8,  0]])
targets
torch.Size([4, 8])
tensor([[46, 43,  1, 58, 47, 51, 43,  1],
        [50,  1, 51, 43,  1, 52, 53, 58],
        [ 1, 49, 52, 53, 58,  1, 49, 52],
        [45, 46, 58, 43, 56,  8,  0, 35]])
when input is [58] the target:46
when input is [58, 46] the target:43
when input is [58, 46, 43] the target:1
when input is [58, 46, 43, 1] the target:58
when input is [58, 46, 43, 1, 58] the target:47
when input is [58, 46, 43, 1, 58, 47] the target:51
when input is [58, 46, 43, 1, 58, 47, 51] the target:43
when input is [58, 46, 43, 1, 58, 47, 51, 43] the target:1
when input is [50] the target:50
when input is [50, 50] the target:1
when input is [50, 50, 1] the target:51
when input is [50, 50, 1, 51] the target:43
when input is [50, 50, 1, 51, 43] the target:1
when input is [50, 50, 1, 51, 43, 1] th

In [54]:
print(xb)

tensor([[58, 46, 43,  1, 58, 47, 51, 43],
        [50, 50,  1, 51, 43,  1, 52, 53],
        [57,  1, 49, 52, 53, 58,  1, 49],
        [59, 45, 46, 58, 43, 56,  8,  0]])


In [63]:
import torch
import torch.nn as nn
from torch.nn import functional as F
vocab_size=100
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, vocab_size)

    def forward(self,idx,targets=None):
        logits=self.token_embedding_table(idx)
        
        if targets is None:
            loss=None
        else:

            B,T,C = logits.shape
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)

        return logits,loss
    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss= self(idx)
            logits= logits[:,-1:]
            probs=F.softmax(logits,dim=-1)
            idx_next=torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx
m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 100])
tensor(5.1629, grad_fn=<NllLossBackward0>)


RuntimeError: prob_dist must be 1 or 2 dim

In [65]:
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [76]:
batch_size=32
for steps in range(10000):
    xb,yb= get_batch('train')

    logits, loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item()) 

2.5147721767425537


In [79]:
print (decode (m. generate (idx = torch.zeros ((1, 1), dtype=torch. long), max_new_tokens=100) [0].tolist()))

RuntimeError: prob_dist must be 1 or 2 dim

In [11]:
l=open('./input.txt','r').read()
print(l[:30])
print(len(l))

Hi there. My name is shravan. 
4566


In [14]:
v=sorted(set(l))
print(v)
print(len(v))

['\t', '\n', ' ', '%', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '8', '9', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'W', 'X', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
68


In [17]:
itos={i:ch for i,ch in enumerate(v)}
stoi={ch:i for i,ch in enumerate(v)}
print(itos)
print(len(itos))
print(stoi)

{0: '\t', 1: '\n', 2: ' ', 3: '%', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '8', 18: '9', 19: 'A', 20: 'B', 21: 'C', 22: 'D', 23: 'F', 24: 'G', 25: 'H', 26: 'I', 27: 'J', 28: 'K', 29: 'L', 30: 'M', 31: 'N', 32: 'O', 33: 'P', 34: 'Q', 35: 'R', 36: 'S', 37: 'T', 38: 'U', 39: 'W', 40: 'X', 41: '_', 42: 'a', 43: 'b', 44: 'c', 45: 'd', 46: 'e', 47: 'f', 48: 'g', 49: 'h', 50: 'i', 51: 'j', 52: 'k', 53: 'l', 54: 'm', 55: 'n', 56: 'o', 57: 'p', 58: 'q', 59: 'r', 60: 's', 61: 't', 62: 'u', 63: 'v', 64: 'w', 65: 'x', 66: 'y', 67: 'z'}
68
{'\t': 0, '\n': 1, ' ': 2, '%': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '8': 17, '9': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'F': 23, 'G': 24, 'H': 25, 'I': 26, 'J': 27, 'K': 28, 'L': 29, 'M': 30, 'N': 31, 'O': 32, 'P': 33, 'Q': 34, 'R': 35, 'S': 36, 'T': 37, 'U': 38, 'W': 39, 'X': 40, '_': 41, 'a': 42, 'b': 43, 'c'

In [32]:
def encode(s):
    return [stoi[ch] for ch in s]
def decode(l):
    return "".join([itos[i] for i in l])

print(len(v))
print(encode("shravan"))
print(decode([60,49,59,42,63,42,55]))
decode(encode("shravan"))

68
[60, 49, 59, 42, 63, 42, 55]
shravan


'shravan'